In [29]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 150)

In [30]:
#../secnect/data/train-logs/failed_login_logs.csv

ssh_folder = "../data/raw-logs/OpenSSH_2k.log_structured.csv"
linux_folder = "../data/raw-logs/Linux_2k.log_structured.csv"

ssh_df = pd.read_csv(ssh_folder)
linux_df = pd.read_csv(linux_folder)

In [31]:
#ssh_df.head(5)
linux_df.head(5)

,LineId,Month,Date,Time,Level,Component,PID,Content,EventId,EventTemplate
0,1,Jun,14,15:16:01,combo,sshd(pam_unix),19939.0,authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4,E16,authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=<*>
1,2,Jun,14,15:16:02,combo,sshd(pam_unix),19937.0,check pass; user unknown,E27,check pass; user unknown
2,3,Jun,14,15:16:02,combo,sshd(pam_unix),19937.0,authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4,E16,authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=<*>
3,4,Jun,15,02:04:59,combo,sshd(pam_unix),20882.0,authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net user=root,E18,authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=<*> user=root
4,5,Jun,15,02:04:59,combo,sshd(pam_unix),20884.0,authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net user=root,E18,authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=<*> user=root


In [32]:
general_drop = ["LineId", "EventId", "EventTemplate"]
ssh_df.drop(columns=general_drop, inplace=True)
linux_df.drop(columns=general_drop, inplace=True)

In [33]:
linux_colums = list(linux_df.columns)
linux_df["text"] = linux_df[linux_colums].astype(str).agg(" ".join, axis=1)
linux_df.drop(columns=linux_colums, inplace=True)
linux_df.head(5)

,text
0,Jun 14 15:16:01 combo sshd(pam_unix) 19939.0 authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4
1,Jun 14 15:16:02 combo sshd(pam_unix) 19937.0 check pass; user unknown
2,Jun 14 15:16:02 combo sshd(pam_unix) 19937.0 authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4
3,Jun 15 02:04:59 combo sshd(pam_unix) 20882.0 authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet....
4,Jun 15 02:04:59 combo sshd(pam_unix) 20884.0 authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet....


In [34]:
ssh_colums = list(ssh_df.columns)
ssh_df["text"] = ssh_df[ssh_colums].astype(str).agg(" ".join, axis=1)
ssh_df.drop(columns=ssh_colums, inplace=True)
ssh_df.head(5)

,text
0,Dec 10 06:55:46 LabSZ 24200 reverse mapping checking getaddrinfo for ns.marryaldkfaczcz.com [173.234.31.186] failed - POSSIBLE BREAK-IN ATTEMPT!
1,Dec 10 06:55:46 LabSZ 24200 Invalid user webmaster from 173.234.31.186
2,Dec 10 06:55:46 LabSZ 24200 input_userauth_request: invalid user webmaster [preauth]
3,Dec 10 06:55:46 LabSZ 24200 pam_unix(sshd:auth): check pass; user unknown
4,Dec 10 06:55:46 LabSZ 24200 pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=173.234.31.186


In [35]:
ssh_df["label"] = np.select(
    [
        ssh_df["text"].str.contains("Failed password for", case=False, na=False),
        ssh_df["text"].str.contains("Accepted password for", case=False, na=False),
    ],
    ["Failed_login", "Success_login"],
    default="Other",
)

linux_df["label"] = np.select(
    [
        linux_df["text"].str.contains("authentication failure;", case=False, na=False),
        linux_df["text"].str.contains("accepted password for | session opened for" , case=False, na=False),
    ],
    ["Failed_login", "Success_login"],
    default="Other",
)


In [36]:
ssh_df['label'].value_counts()
ssh_df['label'].value_counts()

label
Other            1479
Failed_login      520
Success_login       1
Name: count, dtype: int64

In [37]:
linux_df['label'].value_counts()
linux_df.head(20)

linux_100 = linux_df.iloc[0:100]
linux_100.head(100)

ssh_100 = ssh_df.iloc[0:100]
ssh_100.head(100)





,text,label
0,Dec 10 06:55:46 LabSZ 24200 reverse mapping checking getaddrinfo for ns.marryaldkfaczcz.com [173.234.31.186] failed - POSSIBLE BREAK-IN ATTEMPT!,Other
1,Dec 10 06:55:46 LabSZ 24200 Invalid user webmaster from 173.234.31.186,Other
2,Dec 10 06:55:46 LabSZ 24200 input_userauth_request: invalid user webmaster [preauth],Other
3,Dec 10 06:55:46 LabSZ 24200 pam_unix(sshd:auth): check pass; user unknown,Other
4,Dec 10 06:55:46 LabSZ 24200 pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=173.234.31.186,Other
...,...,...
95,Dec 10 07:28:35 LabSZ 24271 Received disconnect from 112.95.230.3: 11: Bye Bye [preauth],Other
96,Dec 10 07:28:35 LabSZ 24273 pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=112.95.230.3 user=root,Other
97,Dec 10 07:28:37 LabSZ 24273 Failed password for root from 112.95.230.3 port 45699 ssh2,Failed_login
98,Dec 10 07:28:37 LabSZ 24273 Received disconnect from 112.95.230.3: 11: Bye Bye [preauth],Other


In [38]:
df_logs_4k = pd.concat([ssh_df, linux_df], ignore_index=True)
df_logs_4k.head(10)

,text,label
0,Dec 10 06:55:46 LabSZ 24200 reverse mapping checking getaddrinfo for ns.marryaldkfaczcz.com [173.234.31.186] failed - POSSIBLE BREAK-IN ATTEMPT!,Other
1,Dec 10 06:55:46 LabSZ 24200 Invalid user webmaster from 173.234.31.186,Other
2,Dec 10 06:55:46 LabSZ 24200 input_userauth_request: invalid user webmaster [preauth],Other
3,Dec 10 06:55:46 LabSZ 24200 pam_unix(sshd:auth): check pass; user unknown,Other
4,Dec 10 06:55:46 LabSZ 24200 pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=173.234.31.186,Other
5,Dec 10 06:55:48 LabSZ 24200 Failed password for invalid user webmaster from 173.234.31.186 port 38926 ssh2,Failed_login
6,Dec 10 06:55:48 LabSZ 24200 Connection closed by 173.234.31.186 [preauth],Other
7,Dec 10 07:02:47 LabSZ 24203 Connection closed by 212.47.254.145 [preauth],Other
8,Dec 10 07:07:38 LabSZ 24206 Invalid user test9 from 52.80.34.196,Other
9,Dec 10 07:07:38 LabSZ 24206 input_userauth_request: invalid user test9 [preauth],Other


In [39]:
from sklearn.utils import shuffle
df_logs_4k_shuffled = shuffle(df_logs_4k)
df_logs_4k_shuffled.head(5)

,text,label
429,Dec 10 09:12:04 LabSZ 24471 pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=103.99.0.122,Other
3673,Jul 22 09:27:24 combo ftpd 12279.0 connection from 211.42.188.206 () at Fri Jul 22 09:27:24 2005,Other
3798,Jul 25 06:39:18 combo ftpd 24968.0 connection from 206.47.209.10 () at Mon Jul 25 06:39:18 2005,Other
3240,Jul 11 11:33:13 combo gdm(pam_unix) 2803.0 check pass; user unknown,Other
2244,Jun 22 04:30:56 combo sshd(pam_unix) 17140.0 check pass; user unknown,Other


In [ ]:
#df_logs_4k_shuffled.to_csv("4k_labeled_logs.csv", index=False)

In [40]:
import pandas as pd
failed_login_logs = pd.read_csv("../data/sample-logs/failed_login_logs.csv")
scc_df = pd.read_csv("../data/sample-logs/successful_login_logs.csv")
fll_df = failed_login_logs
fll_df.head()

,ID,Asset,Log
0,1,Windows Server,"<Event xmlns=""http://schemas.microsoft.com/win/2004/08/events/event""><System><EventID>4625</EventID><TimeCreated SystemTime=""12:00:00Z""/></System>..."
1,2,Ubuntu SSH,May 09 12:04:00 ubuntu sshd[7672]: Failed password for usr001 from 203.0.113.11 port 48734 ssh2
2,3,macOS,2025-05-09 loginwindow[7211]: User login FAILED for user usr002 (UID 586)
3,4,Cisco ASA,"May 09 2025 12:12:00: %ASA-4-113015: AAA user authentication Rejected : user = usr003 , IP = 203.0.113.33"
4,5,Palo Alto GlobalProtect,"2025/05/09 12:16:00,001801000013,SYSTEM,gplogin,0,2025/05/09 12:16:00,203.0.113.44,10.0.0.1,Login,globalprotect,user=usr004,tunnel=yes,result=failure"


In [41]:
fll_df["text"] = fll_df["Log"]
scc_df["text"] = scc_df["Log"]

In [42]:
fll_df["label"] = "Failed_login"
scc_df["label"] = "Success_login"
scc_df.head(5)

,ID,Asset,Log,text,label
0,1,Windows Server,"<Event><System><EventID>4624</EventID></System><EventData><Data Name=""SubjectUserName"">user000</Data><Data Name=""IpAddress"">192.0.2.0</Data></Even...","<Event><System><EventID>4624</EventID></System><EventData><Data Name=""SubjectUserName"">user000</Data><Data Name=""IpAddress"">192.0.2.0</Data></Even...",Success_login
1,2,Ubuntu SSH,May 08 12:03:00 ubuntu sshd[2584]: Accepted password for user001 from 192.0.2.7 port 51763 ssh2,May 08 12:03:00 ubuntu sshd[2584]: Accepted password for user001 from 192.0.2.7 port 51763 ssh2,Success_login
2,3,macOS,2025-05-08 loginwindow[5333]: User login succeeded for user user002 (UID 559),2025-05-08 loginwindow[5333]: User login succeeded for user user002 (UID 559),Success_login
3,4,Cisco ASA,May 08 2025 12:09:00: %ASA-6-722051: VPN Login: User=user003 IP=192.0.2.21 Duration=00:00:00 GroupPolicy=vpngroup,May 08 2025 12:09:00: %ASA-6-722051: VPN Login: User=user003 IP=192.0.2.21 Duration=00:00:00 GroupPolicy=vpngroup,Success_login
4,5,Palo Alto GlobalProtect,"2025/05/08 12:12:00,001801000013,SYSTEM,gplogin,0,2025/05/08 12:12:00,192.0.2.28,10.0.0.1,Login,globalprotect,user=user004,tunnel=yes,result=success","2025/05/08 12:12:00,001801000013,SYSTEM,gplogin,0,2025/05/08 12:12:00,192.0.2.28,10.0.0.1,Login,globalprotect,user=user004,tunnel=yes,result=success",Success_login


In [43]:
fll_df.drop(columns=["ID", "Asset", "Log"], inplace=True)
scc_df.drop(columns=["ID", "Asset", "Log"], inplace=True)
scc_df.head(2)

,text,label
0,"<Event><System><EventID>4624</EventID></System><EventData><Data Name=""SubjectUserName"">user000</Data><Data Name=""IpAddress"">192.0.2.0</Data></Even...",Success_login
1,May 08 12:03:00 ubuntu sshd[2584]: Accepted password for user001 from 192.0.2.7 port 51763 ssh2,Success_login


In [44]:
#linux_100, ssh_100, fll_df, scc_df

df_400logs = pd.concat([linux_100, ssh_100, fll_df, scc_df], ignore_index=True)
df_400logs.to_csv("df_400logs.csv", index=False)